In [6]:
import os
import subprocess
import Bio 
from Bio.Align import MultipleSeqAlignment
import glob
from Bio import SeqRecord
from Bio.Seq import Seq #added
import pandas as pd
import genomicranges as gr
from Bio.Seq import Seq
import pyranges as pr
from Bio.SeqUtils import GC
from Bio import AlignIO, SeqIO
dir(Bio.SeqUtils)

['Alphabet',
 'GC',
 'GC123',
 'GC_skew',
 'IUPACData',
 'MutableSeq',
 'Seq',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'cos',
 'molecular_weight',
 'nt_search',
 'pi',
 'print_function',
 're',
 'seq1',
 'seq3',
 'sin',
 'six_frame_translations',
 'xGC_skew']

In [ ]:
myPath = #PATH TO GITHUB FOLDER

## GC normalization plan
    -need to do this for each human - species file individually (or else the alignment cols may be off) 
    -find regions of human intron that have the same GC content as the promoter (concatenated together) 
    -concatenate those together 
    -can pull out intron from each species/gene pair

In [7]:
def give_me_gc_content_dict(type_of_seq):
    #type of seq = promoter or exon character
    species= ['chimp', 'gorilla', 'orangutan', 'pileatedgibbon', 'marm', 'mac']
    genes = ["EIF1AX", "EIF1AY", "KDM5D" , "KDM5C","UTY", "KDM6A", "ZFY", "ZFX", "DDX3Y" ,"DDX3X", "USP9Y" , "USP9X", "RPS4Y1", "RPS4X"] 

    gc_prom_dict = {}

    for spec in species: 
        for gene in genes: 
            try:
                sample_seqY = AlignIO.read(myPath + "/sequences/primates/" + gene + '/' + 'humanmasked.' + spec + '_' + type_of_seq + '_msa_filtered.phy', "phylip")
                gc_prom_dict[(spec, gene)] = GC(sample_seqY[0].seq)
            except: 
                next
    return gc_prom_dict
                                
gc_prom_dict =     give_me_gc_content_dict('promoter')
print(gc_prom_dict)
gc_exon_dict =     give_me_gc_content_dict('exon')
print(gc_exon_dict)


{}
{}


In [111]:
def give_me_intron_coords(gc_prom_dict):
    
    same_gc_introns = {} #[[1,2,3], [4,5,6]] #starts list and end list
                     #can make these files have the exact same GC content 
 
    species= ['chimp', 'gorilla', 'orangutan', 'pileatedgibbon', 'marm', 'mac']

    genes = ["EIF1AX", "EIF1AY", "KDM5D" , "KDM5C","UTY", "KDM6A", "ZFY", "ZFX", "DDX3Y" ,"DDX3X", "USP9Y" , "USP9X", "RPS4Y1", "RPS4X"] 

    for gene in genes: #only running through sample_seqY 

        for spec in species:
            try:
                sample = AlignIO.read(myPath + "sequences/primates/" + gene + '/' + 'humanmasked.' + spec + '_intron_msa_filtered.phy', "phylip")
                if len(sample) == 1: 
                    print('here', gene, spec)

                    continue 
            except: 
                continue 
                print(gene, spec)
            try:
                target = gc_prom_dict[(spec, gene)]
            except: 
                print(target) 
                continue
            
            range_gc = [target - 5, target + 5]
                
            for s in range(0, len(sample[0].seq), 50):                                                   #take thousand intervals that are +/- 5% of the promoter GC
                e = s + 50
                gc_perc = GC(sample[0][s:e].seq)
                if (range_gc[0] < gc_perc) & (gc_perc < range_gc[1]):
                    try:
                        same_gc_introns[(spec,gene)][0].append(s)
                        same_gc_introns[(spec,gene)][1].append(e)
                    except: 
                        same_gc_introns[(spec,gene)] = [[s],[e]]
                    
    return same_gc_introns

same_gc_introns_promoters = give_me_intron_coords(gc_prom_dict)
same_gc_introns_exons = give_me_intron_coords(gc_exon_dict)


In [8]:
def make_new_intron_files(same_gc_introns, type_of_norm):

    for key, values in same_gc_introns.items():
        gene = key[1]
        spec = key[0]
        print(gene, spec)
        seq1= ""
        seq2 = ""

        sample = AlignIO.read(myPath + "/sequences/primates/" + gene + '/' + 'humanmasked.' + spec + '_intron_msa_filtered.phy', "phylip")

        for ix in range(0, len(values[0])):
            s = same_gc_introns[key][0][ix]
            e = same_gc_introns[key][1][ix] #end list
           # print(s,e)
            new_seq = [seq[s:e] for seq in sample]
            seq1 = seq1+str(new_seq[0].seq)
            seq2 = seq2+str(new_seq[1].seq)

        alignment = MultipleSeqAlignment([])

    # Create individual sequences and add them to the alignment 
        record1 = seq1
        record2 = seq2

        newmsa = MultipleSeqAlignment([])

        new_record1 = SeqRecord.SeqRecord("")
        new_record1.id = "human"
        new_record1.seq = Seq(record1) #added this 
        newmsa.append(new_record1)
        new_record = SeqRecord.SeqRecord("")
        new_record.id = "sequence2"
        new_record.seq = Seq(record2)
        newmsa.append(new_record)
        print(len(record1))
        AlignIO.write(newmsa, myPath + "/sequences/primates/" + gene + "/humanmasked." + spec + "_intron_msa_GCadj" + type_of_norm + ".phy", "phylip")
        
make_new_intron_files(same_gc_introns_exons, 'for_exon')
make_new_intron_files(same_gc_introns_promoters, 'for_promoter')


NameError: name 'same_gc_introns_exons' is not defined